In [42]:
from deltalake import DeltaTable, write_deltalake
import duckdb
con = duckdb.connect()

In [43]:
con.sql(""" CREATE SECRET secret1 (
            TYPE S3,
            KEY_ID 'minio',
            SECRET 'minio123',
            REGION 'us-east-1',
            ENDPOINT 'localhost:9000',
            URL_STYLE 'path',
            USE_SSL 'false'
        );
    """)

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

In [44]:
storage_options = {
        'AWS_ACCESS_KEY_ID': 'minio',
        'AWS_SECRET_ACCESS_KEY': 'minio123',
        "AWS_ENDPOINT_URL": "http://localhost:9000",
        "AWS_allow_http": "true",
        'AWS_REGION': 'us-east-1',
        'AWS_S3_ALLOW_UNSAFE_RENAME': 'true'
}

In [4]:
def escreve_delta(df, tableName, modoEscrita):
    uri = f's3://bronze/delta_estudo/{tableName}'

    write_deltalake(uri,
                    df,
                    mode=modoEscrita,
                    storage_options=storage_options)

In [45]:
def ler_delta(tableName):

    uri = f's3://bronze/delta_estudo/{tableName}'

    dt = DeltaTable(uri, storage_options=storage_options)

    return dt

In [ ]:
df = con.sql("select * from 'c:\\COMEX\\deltalake_files\\categories.csv';").to_df()
df

In [13]:
escreve_delta(df, 'categories', 'append')

In [46]:
dt = ler_delta('categories')

In [ ]:
dt.schema().to_pyarrow()


In [ ]:
dt.to_pandas()

#### Deletando regitros

In [ ]:
dt.delete("category_id > 4")

In [ ]:
dt.to_pandas()

#### Merge

In [ ]:
df = con.sql("select * from 'c:\\COMEX\\deltalake_files\\categories.csv';").to_df()
df

In [ ]:
(
    dt.merge(
        source=df,
        predicate="target.category_id = source.category_id",
        source_alias="source",
        target_alias="target"
    )
    .when_matched_update_all()
    .when_not_matched_insert_all()
    .execute()
)

In [ ]:
dt.to_pandas().sort_values('category_id')

#### Histórico de operações no delta_log

In [ ]:
dt.history() # delta_log json

In [ ]:
prova = con.sql("SELECT * FROM 's3://bronze/delta_estudo/categories/_delta_log/*.json'")
prova['commitInfo']['operation'].aggregate('operation')

#### Manutenção de arquivos deletados

In [ ]:
dt.vacuum() # Apaga arquivos após 7 dias da execução do comando "delete"

In [ ]:
dt.vacuum(dry_run=True) #Padrão (modo de ensaio)

In [ ]:
dt.vacuum(retention_hours=680, dry_run=True, enforce_retention_duration=False)
#dt.vacuum(dry_run=False, enforce_retention_duration=True)  #Não execute, a menos que tenha certeza da exclusão!

In [ ]:
dt.files() # Número de arquivos parquet na tabela delta

#### Time Travel

In [ ]:
dt.version() # versão atual

In [35]:
dt.load_as_version(1) # Ler uma versão anterior

In [ ]:
dt.version()

In [ ]:
dt.to_pandas()

In [ ]:
dt.restore(0, ignore_missing_files=True) # Restaurar uma versão específica.

In [ ]:
dt.to_pandas()

In [ ]:
dt.optimize.compact() # Transforma vários arquivos pequenos em poucos arquivos grandes

In [ ]:
dt.optimize.z_order(["category_id", "category_name"]) # Performance em consultas

In [48]:
con.close()